Nhấn Ctrl+F9 để chạy tất cả

In [1]:
!pip -qq install -U xformers --index-url https://download.pytorch.org/whl/cu118
!pip -qq install langchain optimum qdrant-client wikipedia peft wikipedia accelerate bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 614.6 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.1.1+cu118 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.1+cu118 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.1+cu118 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.1.1+cu118 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.6/808.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setu

In [1]:
GENERATE_MODEL_NAME="phatjk/vietzephyr-7b-lora-8bit"
EMBEDDINGS_MODEL_NAME="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
QDRANT_URL = "<QDRANT_URL>"
QDRANT_COLLECTION_NAME = "nttu_sotay_vector_db_v1"
HUGGINGFACE_API_KEY = "<HUGGINGFACE_API_KEY>"

In [2]:
#@title API KEY OF DATABASE
QDRANT_API_KEY =      "<QDRANT_API_KEY>"

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from optimum.bettertransformer import BetterTransformer
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_rerank = AutoModelForSequenceClassification.from_pretrained('amberoad/bert-multilingual-passage-reranking-msmarco').to(device)
model_rerank = BetterTransformer.transform(model_rerank)
tokenizer_rerank = AutoTokenizer.from_pretrained('amberoad/bert-multilingual-passage-reranking-msmarco')

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [4]:
from langchain.schema.document import Document
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.retrievers import WikipediaRetriever
from typing import List
class RerankRetriever(VectorStoreRetriever):
    vectorstore: VectorStoreRetriever
    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query]*len(candidates)
        features = tokenizer_rerank(queries, candidates,  padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
            # relevant_docs = docs[indices[0]]
        return [docs[indices[0]],docs[indices[1]]]
class RerankWikiRetriever(VectorStoreRetriever):
    vectorstore: WikipediaRetriever
    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query]*len(candidates)
        features = tokenizer_rerank(queries, candidates,  padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
            # relevant_docs = docs[indices[0]]
        return [docs[indices[0]],docs[indices[1]]]

In [5]:
from langchain.retrievers import WikipediaRetriever
from langchain.vectorstores import Qdrant
from langchain.llms import HuggingFacePipeline
from qdrant_client import QdrantClient
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.chains import RetrievalQA,MultiRetrievalQAChain
from langchain.llms import HuggingFaceHub
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
class LLMServe:
    def __init__(self) -> None:
      self.embeddings = self.load_embeddings()
      self.current_source = "wiki"
      self.tokenizer = None
      self.retriever = self.load_retriever(retriever_name = self.current_source,embeddings=self.embeddings)
      self.pipe = self.load_model_pipeline(max_new_tokens=300)
      self.prompt = self.load_prompt_template()
      self.rag_pipeline = self.load_rag_pipeline(llm=self.pipe,
                                            retriever=self.retriever,
                                            prompt=self.prompt)
    def load_embeddings(self):
      embeddings = HuggingFaceInferenceAPIEmbeddings(
          model_name=EMBEDDINGS_MODEL_NAME,
          api_key = HUGGINGFACE_API_KEY,
          model_kwargs = {'device': "auto"}
      )
      return embeddings

    def load_retriever(self,retriever_name,embeddings):
      retriever=None
      if retriever_name == "wiki":
        retriever = RerankWikiRetriever(vectorstore = WikipediaRetriever(lang="vi",
                                       doc_content_chars_max=800,top_k_results=15))
      else:
        client = QdrantClient(
            url=QDRANT_URL,api_key=QDRANT_API_KEY, prefer_grpc=False
        )
        db = Qdrant(client=client,
                    embeddings=embeddings,
                    collection_name=QDRANT_COLLECTION_NAME)
        retriever = RerankRetriever(vectorstore = db.as_retriever(search_kwargs={"k":15}))

      return retriever

    def load_model_pipeline(self,max_new_tokens=100):
      config = PeftConfig.from_pretrained(GENERATE_MODEL_NAME)
      model_gen = AutoModelForCausalLM.from_pretrained(
          config.base_model_name_or_path,
          return_dict=True,
          load_in_8bit=True,
          device_map='auto'
      )
      tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
      self.tokenizer = tokenizer
      # Load the Lora model
      qa_model = PeftModel.from_pretrained(model_gen, GENERATE_MODEL_NAME)

      return HuggingFacePipeline(pipeline=pipeline("text-generation", model=qa_model, tokenizer=tokenizer, max_new_tokens=max_new_tokens))

    def load_prompt_template(self):


      messages = [
        {
            "role": "system",
            "content":"{context}",
        },
        {"role": "user", "content": "{question}"},
      ]
      self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

      # query_template = "Bạn là một trợ lý của trường Đại học Nguyễn Tất Thành. Hãy trả lời câu hỏi sau dựa trên ngữ cảnh, nếu ngữ cảnh không cung cấp câu trả lời hoặc không chắc chắn hãy trả lời 'Tôi không biết thông tin này, tuy nhiên đoạn thông tin dưới phần tham khảo có thể có câu trả lời cho bạn!' đừng cố tạo ra câu trả lời không có trong ngữ cảnh.\nNgữ cảnh: {context} \nCâu hỏi: {question}\nTrả lời: "
      query_template = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
      # query_template = "Bạn là một chatbot thông minh trả lời câu hỏi dựa trên ngữ cảnh (context).\n\n### Context:{context} \n\n### Human: {question}\n\n### Assistant:"
      prompt = PromptTemplate(template=query_template,
                        input_variables= ["context","question"])
      return prompt

    def load_rag_pipeline(self,llm,retriever,prompt):
      rag_pipeline = RetrievalQA.from_chain_type(
      llm=llm, chain_type='stuff',
      retriever=retriever,
      chain_type_kwargs={
      "prompt": prompt
      },
      return_source_documents=True)
      return rag_pipeline

    def rag(self,source):
      if source == self.current_source:
        return self.rag_pipeline
      else:
        self.retriever = self.load_retriever(retriever_name=source,embeddings=self.embeddings)
        self.rag_pipeline = self.load_rag_pipeline(llm=self.pipe,
                                      retriever=self.retriever,
                                      prompt=self.prompt)
        self.current_source = source
        return self.rag_pipeline

In [6]:
app = LLMServe()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

In [7]:
# 1
print(app.rag(source="nttu")("Điều kiện nhận học bổng?")["result"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/optimum/bettertransformer/models/encoder_models.py:301: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, ~attention_mask)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Để đạt học bổng, sinh viên cần học lực và điểm rèn luyện từ loại khá trở lên. Điều kiện nhận học bổng bao gồm:

 1. Sinh viên hoàn thành tất cả các học phần đã đăng ký trong năm học (gồm cả 03 học kỳ trong năm học).
 2. Điểm trung bình chung của năm học đạt từ 2.50 trở lên theo thang điểm 4.
 3. Sinh viên đạt tổng số tín chỉ tích lũy từ 30 tín chỉ trở lên trong năm học, riêng với SV năm cuối đạt tổng số tín chỉ tích lũy từ 20 tín chỉ trở lên trong năm học


In [8]:
# 2
print(app.rag(source="nttu")("Bao nhiêu điểm thì học lực Xuất sắc?")["result"])

Để tìm điểm hệ 10 của học lực Xuất sắc, chúng ta cần xem xét bảng học bổng được cung cấp.

 Bảng học bổng nói rằng học lực Xuất sắc có điểm hệ 10 từ 9.0 đến 10. Vì vậy, điểm hệ 10 của học lực Xuất sắc là 9.0 đến 10.

 Vì vậy, học lực Xuất sắc có điểm hệ 10 từ 9.0 đến 10. Điểm hệ 10 của học lực Xuất sắc là 9.0 đến 10.

 Vì vậy, học lực Xuất sắc có điểm hệ 10 từ 9.0 đến 10. Điểm h


In [9]:
# 3
print(app.rag(source="nttu")("Bao nhiêu điểm thì học lực Giỏi?")["result"])

Học lực Giỏi có điểm hệ 10 từ 8.0 đến 8.9. Điểm hệ 10 được chia thành 4 điểm hệ, vì vậy điểm hệ 4 của học lực Giỏi là từ 3.20 đến 3.59.

 Để tìm số điểm thì học lực Giỏi, chúng ta cần nhân điểm hệ 4 với 10. Vì vậy, điểm thì học lực Giỏi là:

 3.20 * 10 = 32
 3.30 * 10 = 33
 3.40 * 10 = 34
 3.50 * 10 = 35
 3.59 * 10 = 36

 Vì vậy, học lực Giỏi có điểm từ 32 đến 36.

 Vì vậy


In [10]:
# 4
print(app.rag(source="nttu")("Bao nhiêu điểm thì học lực Khá?")["result"])

Học lực Khá có điểm hệ 10 từ 7.0 đến 7.9. Để tìm điểm thì bạn cần chia điểm hệ 10 cho 10 và nhân với điểm hệ của học lực Khá. Ví dụ, điểm hệ 10 cho học lực Khá là 7.5. Để tìm điểm thì chúng ta sẽ chia điểm hệ 10 cho 10 và nhân với 7.5:

 Điểm hệ 10 = 10 * 7.5 = 75

 Vì vậy, điểm thì học lực Khá là 75. Điểm thì học lực Khá có thể khác từ nhau tùy thuộc vào các khóa học và môn học cụ thể. Nhưng trong trường hợp


In [11]:
# 5
print(app.rag(source="nttu")("Điều kiện thực tập tốt nghiệp là gì?")["result"])

Điều kiện thực tập tốt nghiệp là một quy trình được thiết lập bởi các trường đại học để đảm bảo rằng sinh viên có thể thực hiện thực tập tốt nghiệp một cách an toàn và hiệu quả. Các điều kiện thực tập tốt nghiệp có thể bao gồm các yêu cầu như sinh viên phải hoàn thành tối thiểu 60% số tín chỉ trong chương trình đào tạo, không nợ các học phần chuyên ngành và được thông báo cụ thể về các điều kiện thực tập tốt nghiệp từ đầu khóa học hoặc chậm nhất tại th


In [12]:
# 6
print(app.rag(source="nttu")("Học phần đã đăng ký có trạng thái N* là gì?")["result"])

Trong trường hợp sinh viên đã đăng ký học phần nhưng không đóng học phí học phần đã đăng ký sau khi đã hết hạn lần 2, họ sẽ nhận được trạng thái N* (nợ). Điều này có nghĩa là họ phải trả học phí học phần đã đăng ký trước khi học kỳ bắt đầu, ngược lại họ sẽ bị hủy học phần và bị xử lý vi phạm theo Quy chế Công tác sinh viên, bị trừ điểm rèn luyện và không được xét học bổng. Muốn đăng ký học lại học phần này, sinh viên phải tự đăng ký lại trên hệ thống


In [13]:
# 7
print(app.rag(source="nttu")("Điều kiện nào để được xét chuyển trường?")["result"])

Để được xét chuyển trường, sinh viên phải đáp ứng các điều kiện sau:

 1. Không đang là sinh viên trình độ năm 1 hoặc năm cuối, không thuộc diện bị xem xét buộc thôi học và còn đủ thời gian học tập theo quy định;
 2. Sinh viên đạt điều kiện trúng tuyển của chương trình, ngành đào tạo cùng khóa tuyển sinh tại nơi chuyển đến;
 3. Nơi chuyển đến có đủ các điều kiện bảo đảm chất lượng, chưa vượt quá năng lực đào tạo đối với chương trình, ngành đà


In [14]:
# 8
print(app.rag(source="nttu")("Nếu điểm thi kết thúc học phần < 4 thì như thế nào?")["result"])

Nếu điểm thi kết thúc học phần < 4, điểm tổng kết học phần sẽ lấy bằng điểm thi kết thúc học phần. Điều này có nghĩa là sinh viên sẽ bị rớt môn học.

 Điều này là do các điểm thi kết thúc học phần được coi là điểm chính trong việc xác định điểm tổng kết học phần. Nếu sinh viên không đạt được điểm thi kết thúc học phần, họ sẽ không đạt được điểm tổng kết học phần. Điểm tổng kết học phần là một phần quan trọng trong việc xác định x


In [15]:
# 9
print(app.rag(source="nttu")("Phí cấp lại thẻ sinh viên khi bị mất là bao nhiêu?")["result"])

Phí cấp lại thẻ sinh viên khi bị mất là 20.000 đồng/lần. Điều này được đề cập trong các quy trình được cung cấp:

 2.2.15	Xin cấp lại mật khẩu
 2.2.16	Sao y
 2.2.17	Cấp phát thẻ sinh viên

 Trong các quy trình này, được đề cập rõ ràng rằng phí cấp lại thẻ sinh viên là 20.000 đồng/lần. Nó được đề cập trong các quy trình để cung cấp thông tin chi tiết về các phí và thời gian giải quyết cho các yêu cầu cụ thể. Do đó, phí cấp lại thẻ sin


In [16]:
# 10
print(app.rag(source="nttu")("Để đạt loại xuất sắc điểm rèn luyện cần bao nhiêu điểm?")["result"])

Để đạt loại Xuất sắc điểm rèn luyện, bạn cần đạt điểm từ 90 đến 100 điểm. Điểm rèn luyện từ 80 đến dưới 90 điểm sẽ đạt loại rèn luyện là Tốt, điểm rèn luyện từ 65 đến dưới 80 điểm sẽ đạt loại rèn luyện là Khá, điểm rèn luyện từ 50 đến dưới 65 điểm sẽ đạt loại rèn luyện là Trung bình, điểm rèn luyện từ 35 đến dưới 50 điểm sẽ đạt loại rèn luyện là Yếu và điểm rèn luyện dưới 35


In [17]:
# 11
print(app.rag(source="nttu")("Để đạt loại tốt điểm rèn luyện cần bao nhiêu điểm?")["result"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Để đạt loại tốt điểm rèn luyện, bạn cần đạt điểm từ 80 đến 90 điểm. Điểm rèn luyện từ 80 đến dưới 90 điểm sẽ đạt loại rèn luyện là Tốt. Những điểm rèn luyện cao hơn 90 điểm sẽ đạt loại Xuất sắc. Những điểm rèn luyện thấp hơn 80 điểm sẽ đạt loại Khá, và những điểm rèn luyện thấp hơn 65 điểm sẽ đạt loại Trung bình. Những điểm rèn luyện thấp hơn 35 điểm sẽ đạt loại Yếu, và những điể


In [18]:
# 12
print(app.rag(source="nttu")("Nếu sinh viên không đạt ở một học phần, phải làm gì?")["result"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Nếu sinh viên không đạt ở một học phần, họ phải đăng ký và học lại từ đầu học phần đó. Điều này được gọi là" học lại từ đầu" hoặc" học lại từ đầu". Điều này có nghĩa là họ phải tập trung vào học phần đó trong một kỳ học mới, để có thể thi lại và đạt điểm. Nếu sinh viên không đạt điểm trong kỳ học mới, họ sẽ bị rớt môn học. Điều này có nghĩa là họ sẽ không được điểm và không được đưa vào khóa học tiếp theo. Điều n


In [19]:
# 13
print(app.rag(source="nttu")("Lớp trưởng được cấp bao nhiêu tiền?")["result"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Lớp trưởng được cấp 200.000 đồng mỗi tháng trong 10 tháng. Điều này có nghĩa là họ sẽ nhận được tổng cộng 2.000.000 đồng trong một năm.

 Bí thư Chi đoàn kiêm nhiệm Lớp trưởng hoàn thành nhiệm vụ được nhận phụ cấp theo mức 200.000 đồng mỗi tháng trong 10 tháng. Điều này có nghĩa là họ sẽ nhận được tổng cộng 2.000.000 đồng trong một năm.

 Tuy nhiên, nếu họ không hoàn thành nhiệm vụ, họ sẽ không nhận được phụ cấp.

 Chi Hội trưởng


In [20]:
# 14
print(app.rag(source="nttu")("Lớp phó được cấp bao nhiêu tiền?")["result"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Đối với Chi Hội trưởng kiêm nhiệm Lớp phó, họ được nhận phụ cấp theo mức 100.000đ/tháng/cá nhân x 10 tháng. Điều này có nghĩa là họ sẽ nhận được 1.000.000 đồng mỗi năm học. Điều này được thực hiện để đảm bảo rằng họ được đưa ra những điều kiện tốt nhất để thực hiện nhiệm vụ của họ và đảm bảo rằng họ có thể được đảm bảo về tình trạng kinh tế. Như vậy, Chi Hội trưởng kiêm nhiệm Lớp phó được cấp 100.000 đồng mỗi th


In [21]:
# 15
print(app.rag(source="nttu")("Bao nhiêu điểm thì bị rớt?")["result"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Nếu điểm thi kết thúc học phần < 4 thì sinh viên bị rớt môn học. Điểm tổng kết học phần lấy bằng điểm thi kết thúc học phần. Điểm thi kết thúc học phần là điểm của sinh viên trong kỳ thi cuối học phần. Nếu điểm thi kết thúc học phần < 4, điểm tổng kết học phần sẽ là 4. Vì vậy, sinh viên bị rớt môn học khi điểm thi kết thúc học phần < 4.

 Vì vậy, bị rớt môn học khi điểm thi kết thúc học phần < 4.

 Trong trường hợp này, điểm thi kết thú
